# Fluxograma do ácido adípico

## Apresentação 

Esse notebook demonstra a formulação e a solução do balanço material para o processo hipotético do ácido adípico descrito por Murphy (2005, Exemplo 2.15) utlizando [o pacote simbólico Sympy](http://sympy.org/en/index.html).

## Problema proposto

[ácido adípico](http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?cid=196) (C<sub>6</sub>H<sub>10</sub>O<sub>4</sub>) raramente ocorre na natureza, mas aproximadamente 2.5 bilhões de kilogramas são produzidos por ano a partir de matérias-primas petroquímicas principalmente para a produção de nylon.

<img src="http://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Adipic-acid-3D-balls.png/320px-Adipic-acid-3D-balls.png" width = 200px/>

Recentemente tem tido um interesse em produzir [ácido adípico a partir de fontes renováveis](http://www.ihs.com/products/chemical/technology/pep/bio-based-adipic-acid.aspx). [Por exemplo, começando pela glicose](http://pubs.acs.org/doi/abs/10.1021/ja00080a057) (C<sub>6</sub>H<sub>12</sub>O<sub>6</sub>), ácido mucônico (C<sub>6</sub>H<sub>6</sub>O<sub>4</sub>) é produzido pela fermentação com a bactérica geneticamente modificada  e. coli. via a reação

<sup>7</sup>&frasl;<sub>3</sub> C<sub>6</sub>H<sub>12</sub>O<sub>6</sub> + <sup>17</sup>&frasl;<sub>2</sub> O<sub>2</sub> &#10141; C<sub>6</sub>H<sub>6</sub>O<sub>4</sub> + 8 CO<sub>2</sub> + 11 H<sub>2</sub>O

que é subsequentemente hidrogenada para formar ácido adípico 

C<sub>6</sub>H<sub>6</sub>O<sub>4</sub> + 2 H<sub>2</sub>  &#10141; C<sub>6</sub>H<sub>10</sub>O<sub>4</sub>

Murphy (2005, Exemplo 2.15) ilustra um fluxograma hipotético desse processo:

<img src="https://github.com/amandalemette/ENG1818/blob/93e1b8c12584e3f633b12edba50a72fe48aaf69b/Imagens/fluxog.png?raw=true"  width=1000 height=200 />

Negligenciando a _E. coli_, vamos determinar os fluxos necessários para produzir 12000 kg/h de ácido adípico assumindo que a glicose está disponível em 10 mg/mL de solução.

### Variáveis de Processo

VAmos começar criando labels para as correntes do fluxograma, para os componentes e para os avanços de reação. 

As abreviações adotadas para as espécies químicas foram:

* A: ácido adípico 
* C: dióxido de carbono
* G: glicose
* H: hidrogênio
* M: ácido mucônico
* N: nitrogênio
* O: oxigênio
* W: água

e X<sub>1</sub> e X<sub>2</sub> denotam os graus de avanços das reações 1 e 2, respectivamente. 

As variáveis correntes denotam os fluxos mássicos em unidade kg/h. 

Os graus de avanços das reações estão expressos na unidade kg-mol/h.

<img src="https://github.com/amandalemette/ENG1818/blob/db8a927d537e6cea9cc5e5b973b170073b151d3a/Imagens/fluxog2.png?raw=true"  width=1000 height=200 />

In [ ]:
# Import the symbolic algebra package sympy
import sympy as sym

# Extents of reactions 1 and 2
sym.var('X1 X2')

# Stream variables
sym.var('O1 N1')           # Stream 1
sym.var('G2 W2')           # Stream 2
sym.var('H3')              # Stream 3
sym.var('N4 C4')           # Stream 4
sym.var('W5')              # Stream 5
sym.var('A6')              # Stream 6
sym.var('O7 N7 G7 W7')     # Stream 7
sym.var('N8 W8 C8 M8')     # Stream 8
sym.var('M9')              # Stream 9
sym.var('H10 M10')         # Stream 10

Como o fluxograma inclui reações, será necessário incluir pesos moleculares nas expressões de balanço de massa. Para isso reunimos os pesos moleculares de todas as espécies em um dicionário Python indexado pelas espécies químicas.

In [2]:
MW = {
    'A': 146.14,
    'C': 44.01,
    'G': 180.16,
    'H': 2.02,
    'M': 142.11,
    'N': 14.01,
    'O': 16.00,
    'W': 18.02
}

### Especificações

<img src="https://github.com/amandalemette/ENG1818/blob/db8a927d537e6cea9cc5e5b973b170073b151d3a/Imagens/fluxog2.png?raw=true"  width=1000 height=200 />

In [3]:
specs = [
    sym.Eq(A6, 12000),
    sym.Eq(N1/MW['N'], (0.79/0.21)*(O1/MW['O'])),
    sym.Eq(G2, 0.01*W2)
]

### Balanço Material

<img src="https://github.com/amandalemette/ENG1818/blob/db8a927d537e6cea9cc5e5b973b170073b151d3a/Imagens/fluxog2.png?raw=true"  width=1000 height=200 />

In [4]:
mixer1 = [
    sym.Eq(0, O1 - O7),
    sym.Eq(0, N1 - N7),
    sym.Eq(0, G2 - G7),
    sym.Eq(0, W2 - W7)
]

reactor1 = [
    sym.Eq(0,  O7 - MW['O']*(17/2)*X1),
    sym.Eq(0,  N7 - N8),
    sym.Eq(0,  G7 - MW['G']*(7/3)*X1),
    sym.Eq(0, -C8 + MW['C']*8*X1),
    sym.Eq(0, -M8 + MW['M']*X1),
    sym.Eq(0,  W7 - W8 + MW['W']*11*X1)
]

separator = [
    sym.Eq(0, N8 - N4),
    sym.Eq(0, C8 - C4),
    sym.Eq(0, M8 - M9),
    sym.Eq(0, W8 - W5)
]

mixer2 = [
    sym.Eq(0, M9 - M10),
    sym.Eq(0, H3 - H10)
]

reactor2 = [
    sym.Eq(0, H10 - MW['H']*2*X2),
    sym.Eq(0, M10 - MW['M']*X2),
    sym.Eq(0, -A6 + MW['A']*X2)
]

mbals = mixer1 + reactor1 + separator + mixer2 + reactor2

In [ ]:
specs + mbals

### Solução

In [ ]:
soln = sym.solve(mbals + specs)
soln

In [ ]:
for key in soln.keys():
    print(f"{key}: {soln[key]:.2f} kg/h")

## Referências

- Introduction to Chemical Processes : Principles, Analysis, Synthesis by Regina M. Murphy (2005).